In [1]:
from pymongo import MongoClient
from tqdm import tqdm

In [2]:
client = MongoClient()
db = client['rf_test']
col_entries = db['entries']
col_inst = db['inst']
col_inds = db['inds']
col_nouns = db['nouns']

## First look at entries

### What are those?

In [3]:
def list_entries():
    d = col_entries.distinct("type")
    return [x for x in d]

### What do we have the most?

In [8]:
entries = Out[4]

In [12]:
def count_entry():
    entry_count = []
    for entry in entries:
        c = col_entries.find({"type":entry}).count()
        print("{}|{}".format(c,entry))
        entry_count.append((c,entry))
    return entry_count

In [14]:
entry_count = Out[13]

In [17]:
sorted(entry_count, key= lambda x: x[0])[::-1]

[(177928, 'URL'),
 (52303, 'Username'),
 (11210, 'InternetDomainName'),
 (7060, 'Source'),
 (4916, 'GeoEntity'),
 (4661, 'Hashtag'),
 (4238, 'Company'),
 (3700, 'Person'),
 (3307, 'OrgEntity'),
 (2704, 'City'),
 (2616, 'Malware'),
 (2196, 'IndustryTerm'),
 (2049, 'ProvinceOrState'),
 (1917, 'Organization'),
 (1384, 'Product'),
 (923, 'Technology'),
 (861, 'IpAddress'),
 (680, 'Position'),
 (452, 'Facility'),
 (335, 'MalwareSignature'),
 (332, 'FileName'),
 (210, 'Country'),
 (207, 'Region'),
 (126, 'PublishedMedium'),
 (125, 'Hash'),
 (123, 'EmailAddress'),
 (97, 'CyberVulnerability'),
 (66, 'Industry'),
 (48, 'MetaType'),
 (43, 'NaturalFeature'),
 (42, 'AttackVector'),
 (29, 'Feature'),
 (28, 'Topic'),
 (25, 'OperatingSystem'),
 (23, 'Operation'),
 (22, 'TVShow'),
 (18, 'SourceMediaType'),
 (16, 'Religion'),
 (16, 'MedicalCondition'),
 (15, 'EntityList'),
 (15, 'MalwareCategory'),
 (15, 'WinRegKey'),
 (13, 'Holiday'),
 (12, 'Commodity'),
 (12, 'ProgrammingLanguage'),
 (9, 'MarketIndex

In these sources of data, usernames seem interesting.

### Usernames

First, we want to look through what are the usernames we have in the database. To find all usernames, we can do:

In [3]:
def find_all_usernames():
    return [x for x in col_entries.find({"type":"Username"})]

Since there are about 50k usernames, let's just look at a random one:

In [4]:
find_all_usernames()[30]

{'_id': ObjectId('58d9f0e4df3e728bd7e112e7'),
 'created': '2014-07-16T14:44:21.851Z',
 'created_at': '2013-12-09T15:48:56.000Z',
 'curated': 0,
 'domain': 'B_E-R',
 'hits': 40,
 'id': 'LTRvjC',
 'meta_type': 'type:Username',
 'name': '@felicita_andrad',
 'type': 'Username'}

Note the 'id' field. Since this correspond to the author field in the instance, we can use this to count the contributions of the username. Therefore, for each username, we can count or list all of the instance that they are associated with.

In [5]:
def find_inst_by_username(userid):
    return [x for x in col_inst.find({"attributes.authors": userid})]

In [6]:
find_inst_by_username("LTRvjC")

[{'_id': ObjectId('58d9f0b6df3e728bd7de539f'),
  'attributes': {'analyzed': '2015-09-21T17:29:21.094Z',
   'authors': ['LTRvjC'],
   'binning_id': 'GoX8I15DMMc',
   'canonic_id': 'GoX8I15DMMc',
   'document_external_id': '646013299067228160',
   'document_offset': 0,
   'document_url': 'url:https://twitter.com/felicita_andrad/statuses/646013299067228160',
   'entities': ['url:http://www.20minutos.es/noticia/2561290/0/app-store-apple/victima-ciberataque-wechat/xcodeghost/',
    'JDYK14',
    'B_LyO',
    'O8A5tj'],
   'fragment_count': 1,
   'function': 'id',
   'general_negative': 0.0,
   'general_positive': 0.0,
   'indicator': 'ciberataque',
   'meta_type': 'type:CyberAttack',
   'negative': 0.0,
   'positive': 0.0,
   'sentiments': {'activism': 0.0,
    'general_negative': 0.0,
    'general_positive': 0.0,
    'negative': 0.0,
    'positive': 0.0,
    'violence': 0.0},
   'target': ['JDYK14'],
   'target_string': 'EFE Un',
   'topics': ['KPzZAE'],
   'user_data': {'followers_count':

We could also go through the data and see which username "have the most to say". 

In [7]:
def username_ranking():
    usernames_ranks = []
    usernames = find_all_usernames()[:100]
    for u in tqdm(usernames):
        uid = u['id']
        name = u['name']
        count = col_inst.find({"attributes.authors":uid}).count()
        usernames_ranks.append((uid, name, count))
    return sorted(usernames_ranks, key=lambda x: x[2])

On my computer, this process would take around **5 hours** to go through the complete dataset and count up all the usernames, we got about 3 username per seconds. So I only did about the first 100 usernames

In [8]:
u_ranks = username_ranking()

100%|██████████| 100/100 [00:33<00:00,  3.03it/s]


For example, in this small subset, the user with the most instance connected to, is

In [11]:
u_ranks[-1]

('KKYGPH', '@CKLeetwt', 18)

In [12]:
def get_inst_for_user(uid):
    return [x for x in col_inst.find({"attributes.authors":uid})]

#### What does this user talks about?

In [15]:
[x['attributes']['indicator'] for x in get_inst_for_user('KKYGPH')]

['Gameover Zeus',
 'SYNful Knock',
 'malware',
 'Superfish',
 'Carberp',
 'Moose malware may infect Linux - based routers',
 'Duqu2',
 'malware',
 'SpyEye',
 'Dyreza',
 'Dridex Botnet',
 'Dridex',
 'Thunderstrike',
 'RIG exploit kit infects 1 million PCs',
 'Stuxnet',
 'Potao malware used to spy on targets in Ukraine, Russia',
 'TorrentLocker',
 'Trojan']

In [16]:
from nltk.tag import pos_tag

In [17]:
def find_nouns(sentence):
    words = sentence.split()
    tagged = pos_tag(words)
    return [w for w, t in tagged if t == 'NNP']

Here, we will look at everything the user has said, and count all the proper nouns.

In [22]:
def words_from_user(uid):
    insts = get_inst_for_user('KKYGPH')
    sentences = [x['fragment'] for x in insts]
    words = {}
    for s in sentences:
        ns = find_nouns(s)
        for n in ns:
            try:
                words[n] += 1
            except KeyError:
                words[n] = 1
    return words

In [26]:
sorted(((a,b) for a,b in words_from_user('KKYGPH').items()), key=lambda x: x[1])[::-1]

[('Dridex', 3),
 ('|', 3),
 ('Carberp', 2),
 ('Cisco', 2),
 ('RT', 2),
 ('Zeus', 2),
 ('Rt', 2),
 ('#ja….', 1),
 ('#malware', 1),
 ('#cybercrime', 1),
 ('Shifu:', 1),
 ('@icyberfighter:', 1),
 ('#ransomware', 1),
 ('#TorrentLocker', 1),
 ('Zealand)', 1),
 ('ANZ', 1),
 ('Russia', 1),
 ('Ukraine,', 1),
 ('Potao', 1),
 ('Korea', 1),
 ('North', 1),
 ('United', 1),
 ('Stuxnet', 1),
 ('ICYMI:', 1),
 ('RIG', 1),
 ('Symantec', 1),
 ('VB', 1),
 ('Recent', 1),
 ('Czech', 1),
 ('Botnet', 1),
 ('Proofpoint', 1),
 ('Dyreza,', 1),
 ('SophosLabs:', 1),
 ('Europol', 1),
 ('SpyEye', 1),
 ('Aussies', 1),
 ('Cryptolocker', 1),
 ('Australia', 1),
 ('LAST:', 1),
 ('AT', 1),
 ('Foxconn', 1),
 ('Moose', 1),
 ('#Australia', 1),
 ('Hemisphere', 1),
 ('Southern', 1),
 ('Works', 1),
 ('Superfish', 1),
 ("Lenovo's", 1),
 ("Knock'", 1),
 ('@SCMagazineAU:', 1),
 ('Shadowserver', 1),
 ('Knock', 1),
 ('SYNful', 1),
 ('@regsecurity:', 1),
 ('Gameover', 1),
 ('Fbi', 1),
 ('@esecurityp:', 1)]

Uhmm, apparently, this user talks about Dridex more than other. This still needs a lot of improvement. One big improvement this approach can use is to categorize the noun. This will give us some context to what the user is talking about. 

Furthermore, we can compare the user and group them by what they are talking about. This come back to our instance database. For example, let's look at an our Dridex malware

In [46]:
def people_and_instance(indicator):
    c = col_inst.find({"attributes.indicator": indicator})
    c = [x for x in c]
    authors = []
    authors_info = {}
    for entry in c:
        try:
            author = entry['attributes']['authors']
        except KeyError:
            author = None
        authors.append(author)
    for a in tqdm(authors):
        if a:
            en = col_entries.find_one({"id": a[0]})
            authors_info[en['id']] = {"name": en['name']}
    return authors_info

In [29]:
people_and_instance('Dridex')

[{'_id': ObjectId('58d9f0b0df3e728bd7ddaa4a'),
  'attributes': {'analyzed': '2015-04-21T16:09:56.977Z',
   'authors': ['KFXuWH'],
   'binning_id': 'Hv1nOiXKhAF',
   'canonic_id': 'Hv1nOiXKhAF',
   'document_category': 'Cyber',
   'document_external_id': '590547118600912896',
   'document_url': 'url:https://twitter.com/sans_isc/status/590547118600912896',
   'entities': ['Lka-Z7',
    'KeibTe',
    'KewCgR',
    'OIZgvL',
    'url:https://isc.sans.edu/diary/Dridex+Redirecting+to+Malicious+Dropbox+Hosted+File+Via+Google/19609'],
   'fragment_count': 1,
   'function': 'id',
   'general_negative': 0.4090909090909091,
   'general_positive': 0.0,
   'indicator': 'Dridex',
   'meta_type': 'type:CyberAttack',
   'method': 'L37Lyq',
   'negative': 0.0,
   'positive': 0.0,
   'related_entities': ['B_HE4'],
   'related_entities_string': 'Google',
   'sentiments': {'activism': 0.0,
    'general_negative': 0.4090909090909091,
    'general_positive': 0.0,
    'negative': 0.0,
    'positive': 0.0,
  

In [32]:
len(Out[29]) #There are 468 entries about Dridex, sweet !!

468

In [47]:
people_and_instance('Dridex')

100%|██████████| 468/468 [00:33<00:00, 19.11it/s]


{'K05qIa': {'name': '@NiightlyCat'},
 'K17xcT': {'name': '@ScottiAlbertoG'},
 'K5dKsC': {'name': '@submoodle'},
 'K7x5By': {'name': '@quallimited'},
 'K9J08m': {'name': '@jabolins'},
 'K9eq1h': {'name': '@Avosec'},
 'KF3KQS': {'name': '@JA25000'},
 'KF8pTm': {'name': '@ximad'},
 'KF9gR7': {'name': '@Seifreed'},
 'KFK0xQ': {'name': '@bartblaze'},
 'KFK57Q': {'name': '@AdamLangePL'},
 'KFKXsd': {'name': '@UK_Tech_News'},
 'KFKg5Z': {'name': '@InfosecNewsBot'},
 'KFKmZP': {'name': '@Insecurestuff'},
 'KFLYUo': {'name': '@SecurityNews'},
 'KFMDfg': {'name': '@BrianHonan'},
 'KFMJsp': {'name': '@MalwareMustDie'},
 'KFMT7t': {'name': '@upgradeoptions'},
 'KFMk-r': {'name': '@evanderburg'},
 'KFNVB9': {'name': '@wopot'},
 'KFO8a5': {'name': '@ciperovich'},
 'KFPMax': {'name': '@ITDataSecurity'},
 'KFPthV': {'name': '@Cephurs'},
 'KFPuks': {'name': '@c_APT_ure'},
 'KFQ5tn': {'name': '@ioerror'},
 'KFQITr': {'name': '@Security_FAQs'},
 'KFRzvY': {'name': '@lemoine_vincent'},
 'KFSwQj': {'name':

When we have a list of people who talks about an topic, we can iterate through the list of ID, find their sentences and count the noun in those. This way, we could see who are "really interested" in the topic.